# GLP-1 RA Market Access Analysis (France, Germany, UK)
This Colab notebook loads the Excel dataset, performs analysis, and exports Tableau-ready CSVs and charts.

In [ ]:

# Install required packages (only needed in Colab)
!pip install pandas matplotlib openpyxl xlsxwriter


In [ ]:

import pandas as pd
import matplotlib.pyplot as plt

# Upload the Excel file manually in Colab (use left panel > Files > Upload)
excel_path = "glp1_eu_market_access.xlsx"

# Load sheets
countries = pd.read_excel(excel_path, sheet_name="countries")
drugs = pd.read_excel(excel_path, sheet_name="drugs")
prices = pd.read_excel(excel_path, sheet_name="prices")
reimb = pd.read_excel(excel_path, sheet_name="reimbursement")
afford = pd.read_excel(excel_path, sheet_name="affordability")

print(countries.head())


In [ ]:

# Join latest-year prices with reimbursement + GDP
latest_year = prices['year'].max()
summary = prices.query("year == @latest_year").merge(
    reimb[['country','brand','status','access_score']], on=['country','brand'], how='left'
).merge(countries[['country','monthly_gdp_pc']], on='country', how='left')

summary['price_pct_of_monthly_gdp_pc'] = (summary['monthly_price_eur']/summary['monthly_gdp_pc']*100).round(2)
summary.head()


In [ ]:

# Save summary CSV for Tableau
summary.to_csv("tableau_master_latest.csv", index=False)
print("Exported tableau_master_latest.csv")


In [ ]:

# Example charts

weg = summary[summary['brand']=="Wegovy"]
plt.bar(weg['country'], weg['monthly_price_eur'])
plt.title("Wegovy Monthly Price (latest year)")
plt.ylabel("EUR")
plt.show()

oz = summary[summary['brand']=="Ozempic"]
plt.scatter(oz['monthly_gdp_pc'], oz['price_pct_of_monthly_gdp_pc'])
for _, row in oz.iterrows():
    plt.annotate(row['country'], (row['monthly_gdp_pc'], row['price_pct_of_monthly_gdp_pc']))
plt.title("Ozempic Affordability vs GDP per Capita (latest year)")
plt.xlabel("Monthly GDP pc (EUR)")
plt.ylabel("Price % GDP pc")
plt.show()
